# Prepare Calibration Images

In [6]:
from ultralytics import YOLO

MODEL_PATH = "../../../runs/detect/yolov8s_rdd2022_2class7/weights/best.pt"

model = YOLO(MODEL_PATH)

model.export(
    format="onnx",
    imgsz=640,
    opset=13,
    simplify=True
)

Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.5.1+cu121 CPU (13th Gen Intel Core i7-13620H)
Model summary (fused): 73 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '../../../runs/detect/yolov8s_rdd2022_2class7/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)

ONNX: starting export with onnx 1.20.1 opset 13...
ONNX: slimming with onnxslim 0.1.84...
ONNX: export success ✅ 0.8s, saved as '../../../runs/detect/yolov8s_rdd2022_2class7/weights/best.onnx' (42.7 MB)

Export complete (1.1s)
Results saved to /home/saber/GitHub/road_anomaly_detection/runs/detect/yolov8s_rdd2022_2class7/weights
Predict:         yolo predict task=detect model=../../../runs/detect/yolov8s_rdd2022_2class7/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=../../../runs/detect/yolov8s_rdd2022_2class7/weights/best.onnx imgsz=640 data=../../../data/rdd2class_yolo/rdd2class.yaml  
Visualize:       https://netron.app


'../../../runs/detect/yolov8s_rdd2022_2class7/weights/best.onnx'

In [2]:
from pathlib import Path
import random
import shutil

# ================= CONFIG =================
RDD_ROOT = Path("../../../data/combined_annotatedv2")   # adjust path
OUT_DIR = Path("../../../data/calibration_images")
NUM_IMAGES = 250
# =========================================

OUT_DIR.mkdir(parents=True, exist_ok=True)

# Collect all images recursively
all_images = list(RDD_ROOT.rglob("*.jpg"))

print(f"Found {len(all_images)} total images")

# Random sample
sampled = random.sample(all_images, NUM_IMAGES)

# Copy & rename cleanly
for i, img_path in enumerate(sampled):
    dst = OUT_DIR / f"calib_{i:04d}.jpg"
    shutil.copy(img_path, dst)

print(f"✅ Copied {NUM_IMAGES} calibration images to {OUT_DIR}")


Found 25600 total images
✅ Copied 250 calibration images to ../../../data/calibration_images


In [2]:
from pathlib import Path
import cv2

imgs = list(Path("../../../data/calibration_images").glob("*.jpg"))
print("Images:", len(imgs))

img = cv2.imread(str(imgs[0]))
print("Shape:", img.shape)

Images: 250
Shape: (600, 600, 3)


In [3]:
import cv2
import numpy as np
from pathlib import Path

CALIB_DIR = Path("../../../data/calibration_images")
IMG_SIZE = 640

def representative_dataset():
    images = sorted(CALIB_DIR.glob("*.jpg"))

    for img_path in images:
        img = cv2.imread(str(img_path))
        if img is None:
            continue

        # Resize to model input
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

        # Convert to float32 [0,1]
        img = img.astype(np.float32) / 255.0

        # Add batch dimension
        img = np.expand_dims(img, axis=0)

        yield [img]

In [5]:
import tensorflow as tf

SAVED_MODEL_DIR = "/home/saber/GitHub/road_anomaly_detection/runs/detect/yolov8s_rdd2022_2class7/weights/yolo_tf_savedmodel"
OUTPUT_TFLITE = "/home/saber/GitHub/road_anomaly_detection/runs/detect/yolov8s_rdd2022_2class7/weights/best_int8.tflite"

converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)

# 🔥 FULL INT8 quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset

# Force INT8 everywhere
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

with open(OUTPUT_TFLITE, "wb") as f:
    f.write(tflite_model)

print("✅ INT8 TFLite model saved as:", OUTPUT_TFLITE)


I0000 00:00:1770440464.214540   17987 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5809 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
W0000 00:00:1770440464.832767   17987 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1770440464.832779   17987 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2026-02-07 10:31:04.834240: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /home/saber/GitHub/road_anomaly_detection/runs/detect/yolov8s_rdd2022_2class7/weights/yolo_tf_savedmodel
2026-02-07 10:31:04.845434: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2026-02-07 10:31:04.845449: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /home/saber/GitHub/road_anomaly_detection/runs/detect/yolov8s_rdd2022_2class7/weights/yolo_tf_savedmodel
I0000 00:00:17704404

✅ INT8 TFLite model saved as: /home/saber/GitHub/road_anomaly_detection/runs/detect/yolov8s_rdd2022_2class7/weights/best_int8.tflite


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8
2026-02-07 10:35:22.166049: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:4061] Estimated count of arithmetic ops: 30.201 G  ops, equivalently 15.101 G  MACs
